In [1]:
import torch

In [2]:
!ffmpeg

ffmpeg version 6.0-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-liblensfun --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --en

In [3]:
import torch.nn as nn
import torch
import yaml

def load_config(path="default.yaml") -> dict:
    with open(path, "r", encoding="utf-8") as ymlfile:
        cfg = yaml.safe_load(ymlfile)
    return cfg

class VideoClassificationModel(nn.Module):

    def __init__(self, cfg):
        super().__init__()

        self.cfg = cfg
        self.num_classes = cfg['num_classes']
        self.path_torchvideo_model = cfg['path_torchvideo_model']
        self.name_torchvideo_model = cfg['name_torchvideo_model']
        self.n_nodes = cfg["n_nodes"]
        self.freeze_layers = cfg.get("freeze_layers", 1.0) > 0.0
        self.dropout = cfg["dropout"]
        self.base_model = torch.hub.load(self.path_torchvideo_model, self.name_torchvideo_model, pretrained=True)

        if self.freeze_layers:
            print("Freeze layers of pretrained model")
            for name, param in self.base_model.named_parameters():
                param.requires_grad = False

        
        self.base_model.blocks[6].proj = nn.Sequential(nn.Linear(2304, self.n_nodes),
                                                        nn.ReLU(),
                                                        nn.Dropout(self.dropout),
                                                        nn.Linear(self.n_nodes, self.num_classes))

        for name, param in self.base_model.named_parameters():
            print("Layer name: {} - requires_grad: {}".format(name, param.requires_grad))
            
    def forward(self, x):
        x = self.base_model(x)
        return x
        

In [4]:
cfg = load_config()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = VideoClassificationModel(cfg).to(device)

Using cache found in C:\Users\leban/.cache\torch\hub\facebookresearch_pytorchvideo_main


Freeze layers of pretrained model
Layer name: blocks.0.multipathway_blocks.0.conv.weight - requires_grad: False
Layer name: blocks.0.multipathway_blocks.0.norm.weight - requires_grad: False
Layer name: blocks.0.multipathway_blocks.0.norm.bias - requires_grad: False
Layer name: blocks.0.multipathway_blocks.1.conv.weight - requires_grad: False
Layer name: blocks.0.multipathway_blocks.1.norm.weight - requires_grad: False
Layer name: blocks.0.multipathway_blocks.1.norm.bias - requires_grad: False
Layer name: blocks.0.multipathway_fusion.conv_fast_to_slow.weight - requires_grad: False
Layer name: blocks.0.multipathway_fusion.norm.weight - requires_grad: False
Layer name: blocks.0.multipathway_fusion.norm.bias - requires_grad: False
Layer name: blocks.1.multipathway_blocks.0.res_blocks.0.branch1_conv.weight - requires_grad: False
Layer name: blocks.1.multipathway_blocks.0.res_blocks.0.branch1_norm.weight - requires_grad: False
Layer name: blocks.1.multipathway_blocks.0.res_blocks.0.branch1_n

In [5]:
import torch
import os
from pytorchvideo.data.encoded_video import EncodedVideo
from torchvision.transforms import Compose, Lambda, Resize
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)

import numpy as np
import pathlib

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors. 
    """
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // 4
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list


class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, cfg, dataset_path, labelclass) -> None:
        super().__init__()
        self.labelclass=labelclass
        self.dataset_path = dataset_path
        self.cfg = cfg
        side_size = 256
        mean = [0.45, 0.45, 0.45]
        std = [0.225, 0.225, 0.225]
        crop_size = 256
        num_frames = cfg["num_frames"]
        #sampling_rate = cfg["sampling_rate"]
        #frames_per_second = cfg["frames_per_second"]
        do_crop_video = cfg.get("crop_video", 1.0) > 0.0

        if do_crop_video:
            print("do_crop_video TRUE")
            self.transform = ApplyTransformToKey(
                key="video",
                transform=Compose(
                    [
                        UniformTemporalSubsample(num_frames),
                        Lambda(lambda x: x/255.0),
                        NormalizeVideo(mean, std),
                        ShortSideScale(size=side_size),
                        CenterCropVideo(crop_size=(crop_size, crop_size)),
                        PackPathway()
                    ]
                ),
            )
        else:
            print("do_crop_video FALSE")
            self.transform = ApplyTransformToKey(
                key="video",
                transform=Compose(
                    [
                        UniformTemporalSubsample(num_frames),
                        Lambda(lambda x: x/255.0),
                        NormalizeVideo(mean, std),
                        #Resize((crop_size, crop_size)),
                        #ShortSideScale(size=side_size),
                        PackPathway()
                    ]
                ),
            )

    def __len__(self):
        return len(list(pathlib.Path(self.dataset_path).rglob("*.mp4")))

    def __getitem__(self, idx):
        video_path = str(list(pathlib.Path(self.dataset_path).rglob("*.mp4"))[idx])
        label = self.labelclass[list(pathlib.Path(self.dataset_path).rglob("*.mp4"))[idx].parent.name]

        if not os.path.exists(video_path):
            print("The video '{}' does not exist ".format(video_path))

        video = EncodedVideo.from_path(video_path)
        start_time = 0
        clip_duration = int(video.duration)
        end_sec = (cfg["num_frames"] * cfg["sampling_rate"])/cfg["frames_per_second"]
        video_data = video.get_clip(start_sec=start_time, end_sec=end_sec)
        video_data = self.transform(video_data)
        inputs = video_data["video"]

        return inputs, label
     


C:\Users\leban\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
C:\Users\leban\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [22]:
path = "C:\\Users\\leban\\Hahaton\\grouped"
labels = set([p.parent.name for p in list(pathlib.Path(path).rglob("*.mp4"))])
labelclass=dict()
i=0
for label in labels:
    labelclass[label]=i
    i+=1 #не очень умно, но время 2 ночи, а во мне полторы бутылки сидра

class2label = dict((v,k) for k,v in labelclass.items())

labelclass.update(class2label)

dataset = ClassificationDataset(cfg,path,labelclass)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
labels

do_crop_video TRUE


{'cartwheel',
 'catch',
 'clap',
 'climb',
 'dive',
 'draw_sword',
 'dribble',
 'fencing',
 'flic_flac',
 'golf',
 'handstand',
 'hit',
 'jump',
 'pick',
 'pour',
 'pullup',
 'push',
 'pushup',
 'shoot_ball',
 'sit',
 'situp',
 'swing_baseball',
 'sword_exercise',
 'throw'}

In [23]:
from torch.utils.data import DataLoader



batch_size = cfg['batch_size']
classification_dataloader_train = DataLoader(dataset=train_dataset,
                                                 batch_size=batch_size,
                                                 shuffle=True,
                                                 num_workers=0,
                                                 drop_last=True)
classification_dataloader_test = DataLoader(dataset=test_dataset,
                                                 batch_size=batch_size,
                                                 shuffle=True,
                                                 num_workers=0,
                                                 drop_last=True)

In [16]:
from tqdm import tqdm
import numpy as np
import random

post_act = torch.nn.Softmax(dim=1)

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def train_batch(inputs, labels, model, optimizer, criterion):
    model.train()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()


@torch.no_grad()
def accuracy(inputs, labels, model):
    model.eval()
    outputs = model(inputs)
    # Get the predicted classes
    preds = post_act(outputs)
    #print('preds: ', preds)
    _, pred_classes = torch.max(preds, 1)
    #print('pred_classes: ', pred_classes)
    #print('labels:', labels)
    is_correct = pred_classes == labels
    #print('is_correct: ', is_correct)
    return is_correct.cpu().numpy().tolist()


@torch.no_grad()
def val_loss(inputs, labels, model, criterion):
    model.eval()
    outputs = model(inputs)
    val_loss = criterion(outputs, labels)
    return val_loss.item()


def train_model(device,
                model,
                criterion,
                optimizer,
                lr_scheduler,
                classification_dataloader_train,
                classification_dataloader_val,
                best_epoch,
                num_epoch,
                best_val_epoch_accuracy,
                checkpoint_dir,
                saving_dir_experiments,
                logger,
                epoch_start_unfreeze=None,
                block_start_unfreeze=None):

    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    print("Start training")
    freezed = True
    for epoch in range(best_epoch, num_epoch):
        logger.log(f'Epoch {epoch}/{num_epoch - 1}')

        if epoch_start_unfreeze is not None and epoch >= epoch_start_unfreeze and freezed:
            print("****************************************")
            print("unfreeze the base model weights")

            if block_start_unfreeze is not None:
                print("unfreeze the layers greater and equal to unfreezing_block: ", block_start_unfreeze)
                #in this case unfreeze only the layers greater and equal the unfreezing_block layer
                for name, param in model.named_parameters():
                    if int(name.split(".")[2]) >= block_start_unfreeze:
                        param.requires_grad = True
            else:
                # in this case unfreeze all the layers of the model
                print("unfreeze all the layer of the model")
                for name, param in model.named_parameters():
                    param.requires_grad = True
            freezed = False

            for name, param in model.named_parameters():
                print("Layer name: {} - requires_grad: {}".format(name, param.requires_grad))
            print("****************************************")

        # define empty lists for the values of the loss and the accuracy of train and validation obtained in the batch of the current epoch
        # then at the end I take the average and I get the final values of the whole era
        train_epoch_losses, train_epoch_accuracies = [], []
        val_epoch_losses, val_epoch_accuracies = [], []

        # iterate on all train batches of the current epoch by executing the train_batch function
        for inputs, labels in tqdm(classification_dataloader_train, desc=f"epoch {str(epoch)} | train"):
            inputs = [i.to(device, non_blocking=True) for i in inputs]
            labels = labels.to(device, non_blocking=True)
            batch_loss = train_batch(inputs, labels, model, optimizer, criterion)
            train_epoch_losses.append(batch_loss)
        train_epoch_loss = np.array(train_epoch_losses).mean()

        # iterate on all train batches of the current epoch by calculating their accuracy
        for inputs, labels in tqdm(classification_dataloader_train, desc=f"epoch {str(epoch)} | train"):
            inputs = [i.to(device, non_blocking=True) for i in inputs]
            labels = labels.to(device, non_blocking=True)
            is_correct = accuracy(inputs, labels, model)
            train_epoch_accuracies.extend(is_correct)
        train_epoch_accuracy = np.mean(train_epoch_accuracies)

        # iterate on all batches of val of the current epoch by calculating the accuracy and the loss function
        for inputs, labels in tqdm(classification_dataloader_val, desc=f"epoch {str(epoch)} | val"):
            inputs = [i.to(device, non_blocking=True) for i in inputs]
            labels = labels.to(device)

            val_is_correct = accuracy(inputs, labels, model)
            val_epoch_accuracies.extend(val_is_correct)
        val_epoch_accuracy = np.mean(val_epoch_accuracies)

        phase = 'train'
        logger.log(f'{phase} LR: {lr_scheduler.get_last_lr()} - Loss: {train_epoch_loss:.4f} - Acc: {train_epoch_accuracy:.4f}')
        phase = 'val'
        print("Epoch: {} - LR:{} - Train Loss: {:.4f} - Train Acc: {:.4f} - Val Acc: {:.4f}".format(int(epoch), lr_scheduler.get_last_lr(), train_epoch_loss, train_epoch_accuracy,val_epoch_accuracy))
        logger.log("-----------")

        train_losses.append(train_epoch_loss)
        train_accuracies.append(train_epoch_accuracy)
        val_accuracies.append(val_epoch_accuracy)

        if best_val_epoch_accuracy < val_epoch_accuracy:
            print("We have a new best model! Save the model")

            #update best_val_epoch_accuracy
            best_val_epoch_accuracy = val_epoch_accuracy
            save_obj = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': lr_scheduler.state_dict(),
                'epoch': epoch,
                'best_eva_accuracy': best_val_epoch_accuracy
            }
            print("Save best checkpoint at: ", os.path.join(checkpoint_dir, 'best.pth'))
            torch.save(save_obj, os.path.join(checkpoint_dir, 'best.pth'),  _use_new_zipfile_serialization=False)
            print("Save checkpoint at: ", os.path.join(checkpoint_dir, 'checkpoint_' + str(epoch) + '.pth'))
            torch.save(save_obj, os.path.join(checkpoint_dir, 'checkpoint_' + str(epoch) + '.pth'),  _use_new_zipfile_serialization=False)

        else:
            print("Save the current model")

            save_obj = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': lr_scheduler.state_dict(),
                'epoch': epoch,
                'best_eva_accuracy': best_val_epoch_accuracy
            }
            print("Save checkpoint at: ", os.path.join(checkpoint_dir, 'checkpoint_' + str(epoch) + '.pth'))
            torch.save(save_obj, os.path.join(checkpoint_dir, 'checkpoint_' + str(epoch) + '.pth'),  _use_new_zipfile_serialization=False)

        lr_scheduler.step()
        torch.cuda.empty_cache()
        print("---------------------------------------------------------")

    print("End training")
    return


In [17]:
checkpoint = None
best_epoch = 0
best_val_epoch_accuracy = 0

dataset_path = cfg['dataset_path']

saving_dir_experiments = cfg['saving_dir_experiments']
saving_dir_model = cfg['saving_dir_model']

num_epoch = cfg['num_epoch']
learning_rate = cfg['learning_rate']
scheduler_step_size = cfg['scheduler_step_size']
scheduler_gamma = cfg['scheduler_gamma']
epoch_start_unfreeze = cfg.get("epoch_start_unfreeze", None)
block_start_unfreeze = cfg.get("block_start_unfreeze", None)

# 2 -  create the directory to save the results and checkpoints
print("Create the project structure")
print("saving_dir_experiments: ", saving_dir_experiments)
saving_dir_model = os.path.join(saving_dir_experiments, saving_dir_model)
print("saving_dir_model: ", saving_dir_model)
os.makedirs(saving_dir_experiments, exist_ok=True)
os.makedirs(saving_dir_model, exist_ok=True)

checkpoint_dir = saving_dir_model

Create the project structure
saving_dir_experiments:  model_experiments_v1
saving_dir_model:  model_experiments_v1\models_augmented_v1


In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)
if checkpoint is not None:
    print('Load the optimizer from the last checkpoint')
    optimizer.load_state_dict(checkpoint['optimizer'])
    exp_lr_scheduler.load_state_dict(checkpoint["scheduler"])

    print('Latest epoch of the checkpoint: ', checkpoint['epoch'])
    print('Setting the new starting epoch: ', checkpoint['epoch'] + 1)
    best_epoch = checkpoint['epoch'] + 1

    print('Setting best_val_epoch_accuracy from checkpoint: ', checkpoint['best_eva_accuracy'])
    best_val_epoch_accuracy = checkpoint['best_eva_accuracy']

class Logger():
    def log(self, text): print(text)

train_model(device=device,
            model=model,
            criterion=criterion,
            optimizer=optimizer,
            lr_scheduler=exp_lr_scheduler,
            classification_dataloader_train=classification_dataloader_train,
            classification_dataloader_val=classification_dataloader_test,
            best_epoch=best_epoch,
            num_epoch=num_epoch,
            best_val_epoch_accuracy=best_val_epoch_accuracy,
            checkpoint_dir=checkpoint_dir,
            saving_dir_experiments=saving_dir_experiments,
            logger=Logger(),
            epoch_start_unfreeze=epoch_start_unfreeze,
            block_start_unfreeze=block_start_unfreeze,)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")


Start training
Epoch 0/9


epoch 0 | train: 100%|█████████████████████████████████████████████████████████████████| 50/50 [05:38<00:00,  6.77s/it]
epoch 0 | train: 100%|█████████████████████████████████████████████████████████████████| 50/50 [05:38<00:00,  6.77s/it]
epoch 0 | val: 100%|███████████████████████████████████████████████████████████████████| 12/12 [01:20<00:00,  6.72s/it]


train LR: [0.001] - Loss: 0.4604 - Acc: 0.9219
Epoch: 0 - LR:[0.001] - Train Loss: 0.4604 - Train Acc: 0.9219 - Val Acc: 0.8802
-----------
We have a new best model! Save the model
Save best checkpoint at:  model_experiments_v1\models_augmented_v1\best.pth
Save checkpoint at:  model_experiments_v1\models_augmented_v1\checkpoint_0.pth
---------------------------------------------------------
Epoch 1/9


epoch 1 | train: 100%|█████████████████████████████████████████████████████████████████| 50/50 [05:39<00:00,  6.79s/it]
epoch 1 | train: 100%|█████████████████████████████████████████████████████████████████| 50/50 [05:36<00:00,  6.73s/it]
epoch 1 | val: 100%|███████████████████████████████████████████████████████████████████| 12/12 [01:21<00:00,  6.78s/it]


train LR: [0.001] - Loss: 0.3233 - Acc: 0.9350
Epoch: 1 - LR:[0.001] - Train Loss: 0.3233 - Train Acc: 0.9350 - Val Acc: 0.8750
-----------
Save the current model
Save checkpoint at:  model_experiments_v1\models_augmented_v1\checkpoint_1.pth
---------------------------------------------------------
Epoch 2/9


...

train LR: [1e-05] - Loss: 0.0360 - Acc: 0.9994
Epoch: 7 - LR:[1e-05] - Train Loss: 0.0360 - Train Acc: 0.9994 - Val Acc: 0.9089
-----------
We have a new best model! Save the model
Save best checkpoint at:  model_experiments_v1\models_augmented_v1\best.pth
Save checkpoint at:  model_experiments_v1\models_augmented_v1\checkpoint_7.pth
---------------------------------------------------------
Epoch 8/9


epoch 8 | train: 100%|█████████████████████████████████████████████████████████████████| 50/50 [05:42<00:00,  6.85s/it]
epoch 8 | train: 100%|█████████████████████████████████████████████████████████████████| 50/50 [05:44<00:00,  6.90s/it]
epoch 8 | val: 100%|███████████████████████████████████████████████████████████████████| 12/12 [01:22<00:00,  6.89s/it]


train LR: [1e-05] - Loss: 0.0270 - Acc: 0.9994
Epoch: 8 - LR:[1e-05] - Train Loss: 0.0270 - Train Acc: 0.9994 - Val Acc: 0.9036
-----------
Save the current model
Save checkpoint at:  model_experiments_v1\models_augmented_v1\checkpoint_8.pth
---------------------------------------------------------
Epoch 9/9


epoch 9 | train: 100%|█████████████████████████████████████████████████████████████████| 50/50 [05:41<00:00,  6.84s/it]
epoch 9 | train: 100%|█████████████████████████████████████████████████████████████████| 50/50 [05:42<00:00,  6.85s/it]
epoch 9 | val: 100%|███████████████████████████████████████████████████████████████████| 12/12 [01:23<00:00,  6.99s/it]


train LR: [1.0000000000000002e-06] - Loss: 0.0221 - Acc: 0.9994
Epoch: 9 - LR:[1.0000000000000002e-06] - Train Loss: 0.0221 - Train Acc: 0.9994 - Val Acc: 0.9089
-----------
Save the current model
Save checkpoint at:  model_experiments_v1\models_augmented_v1\checkpoint_9.pth
---------------------------------------------------------
End training
-------------------------------------------------------------------
-------------------------------------------------------------------


In [28]:
with torch.no_grad():
    for inputs, labels in classification_dataloader_test: # or i, image in enumerate(dataset)
            model.eval()
            outputs = model([i.to(device, non_blocking=True) for i in inputs])
            preds = post_act(outputs)
            _, pred_classes = torch.max(preds, 1)
            print(pred_classes)
            print([class2label[cl.item()] for cl in pred_classes])
            print(labels)
            print([class2label[cl.item()] for cl in labels])
            print((pred_classes == labels.to(device)))

tensor([19, 10,  4, 19,  4,  7,  4, 21, 23,  2, 14, 21,  7, 18, 17,  4, 11,  5,
        18, 17, 22, 23, 12, 12, 15, 10, 21, 18,  0, 12, 11, 23],
       device='cuda:0')
['fencing', 'shoot_ball', 'pick', 'fencing', 'pick', 'sword_exercise', 'pick', 'handstand', 'dribble', 'climb', 'dive', 'handstand', 'sword_exercise', 'golf', 'draw_sword', 'pick', 'clap', 'cartwheel', 'golf', 'draw_sword', 'swing_baseball', 'dribble', 'pullup', 'pullup', 'hit', 'shoot_ball', 'handstand', 'golf', 'flic_flac', 'pullup', 'clap', 'dribble']
tensor([19, 10,  4, 19,  4,  7,  4, 21, 23,  2, 14, 21, 17, 18, 17,  4, 11,  5,
        18, 17, 22, 23, 12, 12, 15, 10, 21, 18,  0, 12, 11, 23])
['fencing', 'shoot_ball', 'pick', 'fencing', 'pick', 'sword_exercise', 'pick', 'handstand', 'dribble', 'climb', 'dive', 'handstand', 'draw_sword', 'golf', 'draw_sword', 'pick', 'clap', 'cartwheel', 'golf', 'draw_sword', 'swing_baseball', 'dribble', 'pullup', 'pullup', 'hit', 'shoot_ball', 'handstand', 'golf', 'flic_flac', 'pull

In [25]:
torch.save(model.state_dict(), 'model_weights.pth')

In [27]:
torch.save(model, 'model.pth')

In [29]:
device = torch.device("cuda")
m = torch.load("model.pth", map_location=device)

In [30]:
with torch.no_grad():
    for inputs, labels in classification_dataloader_test: # or i, image in enumerate(dataset)
            m.eval()
            outputs = m([i.to(device, non_blocking=True) for i in inputs])
            preds = post_act(outputs)
            _, pred_classes = torch.max(preds, 1)
            print(pred_classes)
            print([class2label[cl.item()] for cl in pred_classes])
            print(labels)
            print([class2label[cl.item()] for cl in labels])
            print((pred_classes == labels.to(device)))

tensor([15,  5, 15,  5, 22, 16,  5,  3,  9, 10,  0,  0,  3, 22,  0, 10,  0, 13,
         0, 13,  1, 20,  7,  9,  8, 18, 15, 14, 16, 14, 12,  1],
       device='cuda:0')
['hit', 'cartwheel', 'hit', 'cartwheel', 'swing_baseball', 'sit', 'cartwheel', 'jump', 'pushup', 'shoot_ball', 'flic_flac', 'flic_flac', 'jump', 'swing_baseball', 'flic_flac', 'shoot_ball', 'flic_flac', 'pour', 'flic_flac', 'pour', 'catch', 'throw', 'sword_exercise', 'pushup', 'push', 'golf', 'hit', 'dive', 'sit', 'dive', 'pullup', 'catch']
tensor([15,  5, 15,  5, 22, 16,  5,  3,  9, 10,  0,  0,  3, 22,  0, 10,  0, 13,
         0, 13,  1, 20,  7,  9,  8, 18, 15, 14, 16, 14, 12,  1])
['hit', 'cartwheel', 'hit', 'cartwheel', 'swing_baseball', 'sit', 'cartwheel', 'jump', 'pushup', 'shoot_ball', 'flic_flac', 'flic_flac', 'jump', 'swing_baseball', 'flic_flac', 'shoot_ball', 'flic_flac', 'pour', 'flic_flac', 'pour', 'catch', 'throw', 'sword_exercise', 'pushup', 'push', 'golf', 'hit', 'dive', 'sit', 'dive', 'pullup', 'catch']
